Nice article for inspirations on visualizations: https://ibexorigin.medium.com/yes-these-unbelievable-masterpieces-are-created-with-matplotlib-2256a4c54b12

In [ ]:
# !pip install pandas-profiling
# !pip install kaggle --upgrade
# !pip install ipywidgets --upgrade
# !pip install jupyter --upgrade
# !pip install IProgress --upgrade
# !pip install dask --upgrade
# !jupyter nbextension enable --py widgetsnbextension


!kaggle competitions download -c learnplatform-covid19-impact-on-digital-learning

     |████████████████████████████████| 1.0 MB 32.4 MB/s 
     |████████████████████████████████| 119 kB 74.2 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
from pathlib import Path
from pandas_profiling import ProfileReport
data_dir = Path("/work/input/learnplatform-covid19-impact-on-digital-learning")


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        pass
        # print(os.path.join(dirname, filename))

# This is a new line

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# products_df = pd.read_csv(data_dir/"products_info.csv")
# products_df['Sector(s)'].unique()

# ProfileReport(products_df, title="Our Profile")

<div style="background-color: #f0f0f0; padding: 10px; border: 2px solid #4a4a4a; border-radius: 5px">
<h2 style="color: #3b3b3b">Available data 🗓📌</h2>

<h3 style="color: #3b3b3b">What does the data include?</h3>

- Two measures of learning engagement  (`pct_access` and `engagement_index`) aggregated over **234** US school districts.
- The top **372** ed-tech products (out of **10000**) identified by the Chrome browser extension [learnplatform](https://learnplatform.com/).
- Information about the **234** school districts - heavily anonymized with sideeffects of introduced ranges (instead of actual values) and a lot of missing values.

<h2 style="color: #3b3b3b">Understanding the engagement data 🤓📚</h2>

<h3 style="color: #3b3b3b">Given definitions</h3> 

- `pct_access`: Percentage of students in the district have at least one page-load event of a given product and on a given day
    
    - **Example**: 15% of students in a district in Utah engaged at least once with  on Monday April 14.
    - **Question**: Some numbers are > 1. What should that mean?
    
- `engagement_index`: Total page-load events per one thousand students of a given product and on a given day
    
    - **In simple terms**: Shows how active the students are in the district.
    - **Example**: 341 engagement events per 1000 students were recorded in Chicago on Monday April 14.

<h2 style="color: #3b3b3b">Understanding the product data 🤓📚</h2>

<h3 style="color: #3b3b3b">Sector(s)</h3>

- **Issues**: 
    - What does the sector `PreK-12; Higher Ed; Corporate` exactly cover?
    - `LC - Study Tools` contains Grammarly, `Other` contains `Facebook` -- check relevance for each hypothesis!
         

</div>

## Load Data

In [ ]:
import glob
import pandas as pd 
import dask.dataframe as dd

# engagement_data_sample = pd.read_csv(data_dir / "engagement_data/1000.csv")

all_file_names = list(data_dir.glob("engagement_data/*.csv"))

data_of_district = []

for filename in all_file_names:
    # df = pd.read_csv(filename, index_col=None, header=0)
    df = dd.read_csv(filename, header=0, assume_missing=True)
    district_id = filename.stem
    df["district_id"] = district_id
    data_of_district.append(df)
    
engagement = dd.multi.concat(data_of_district)
# del data_of_district
engagement = engagement.reset_index(drop=True)
engagement.head()

products = dd.read_csv(data_dir / "products_info.csv")
print(engagement['lp_id'].nunique().compute())
print(products['LP ID'].nunique().compute())

districts = dd.read_csv(data_dir /"districts_info.csv")
districts.head()

8646
372


,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
0,8815,Illinois,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000["
1,2685,NaN,NaN,NaN,NaN,NaN,NaN
2,4921,Utah,Suburb,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[","[6000, 8000["
3,3188,NaN,NaN,NaN,NaN,NaN,NaN
4,2238,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import sys
sys.getsizeof(engagement)

64

In [ ]:
_deepnote_run_altair(engagement_df, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"lp_id","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"COUNT(*)","type":"quantitative","sort":null,"aggregate":"count","scale":{"type":"linear","zero":true}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
districts.shape()

NameError: name 'districts' is not defined

In [ ]:
merged_data = pd.merge(products, engagement, left_on = 'LP ID', right_on = 'lp_id')
merged_data['district_id'] = merged_data['district_id'].astype('int64')
merged_data = pd.merge(merged_data, districts, on = 'district_id')
merged_data.drop(['URL', 'lp_id', 'state_abb'], axis = 1, inplace = True)
merged_data.head(3)

NameError: name 'districts' is not defined

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
st_acсess = merged_data.groupby(['state', 'time']).agg({'pct_access': 'mean'}).reset_index()
st_eng = merged_data.groupby(['state', 'time']).agg({'engagement_index': 'mean'}).reset_index()
loc_acсess = merged_data.groupby(['locale', 'time']).agg({'pct_access': 'mean'}).reset_index()
loc_eng = merged_data.groupby(['locale', 'time']).agg({'engagement_index': 'mean'}).reset_index()
cat_acсess = merged_data.groupby(['Basic_category', 'time']).agg({'pct_access': 'mean'}).reset_index()
cat_eng = merged_data.groupby(['Basic_category', 'time']).agg({'engagement_index': 'mean'}).reset_index()

for i in [st_acсess, st_eng, loc_acсess, loc_eng, cat_acсess, cat_eng]:
    i['day_of_week'] = i['time'].dt.dayofweek
    
st_acсess.head(3)

In [ ]:
fig = px.line(st_acсess, x="time", y="pct_access", color="state", line_group="state")

fig.update_layout(plot_bgcolor = 'white', title = 'Dynamics of pct_access of all products by states', 
                  title_font_family = 'monospace', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)

fig.add_vline(x = '2020-03-11', line_width = 3, line_color="red")

fig.add_annotation(
        x='2020-03-11',
        y=2.7,
        text="WHO has declared Covid-19 a pandemic",
        showarrow=True,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            ),
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax= 130,
        ay=1
        )

fig.add_vrect(x0="2020-06-01", x1="2020-08-31", fillcolor="yellow", opacity=0.25, line_width=0)

fig.add_annotation(
        x='2020-07-15',
        y=2.25,
        text="Summer holidays",
        showarrow=False,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            )
        )

fig.update_traces(line_width=1)

fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=86d79b39-1369-4ff2-a556-9a23d285d9c0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>